<a href="https://colab.research.google.com/github/NHV4546/Skin-Cancer-Project/blob/main/Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/NHV4546/Skin-Cancer-Project.git

Cloning into 'Skin-Cancer-Project'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [12]:
print("Trial commit")

Trial commit


In [27]:
from google.colab import userdata
token = userdata.get('token')
repo_url = f"https://{token}@github.com/NHV4546/Skin-Cancer-Project.git"
%cd /content/Skin-Cancer-Project
!git config --global user.name "NHV4546"
!git config --global user.email "dinasamat0110@gmail.com"
!git remote set-url origin $repo_url

/content/Skin-Cancer-Project
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


In [28]:
print("trial2")

trial2


In [29]:
!git add .
!git commit -m "Trial"
!git push origin main


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date
